In [58]:
!pip install nltk sklearn

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [59]:
import nltk
import pandas as pd
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as sklearn_stopwords
nltk_stopwords.words('english')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [60]:
# Combine stopwords into a single set
all_stopwords = set(nltk_stopwords.words('english')).union(set(sklearn_stopwords))
print(f"Total number of unique stopwords from NLTK and scikit-learn: {len(all_stopwords)}")

Total number of unique stopwords from NLTK and scikit-learn: 397


In [86]:
from collections import Counter
corpus_names = [name for name in dir(nltk.corpus) if not name.startswith('_')]
top20 = []
results = []

for name in corpus_names:
    try:
        corpus = getattr(nltk.corpus, name)
        if hasattr(corpus, 'words'):
            raw_count = len(corpus.words())
            fil_corpus = [word for word in corpus.words() if word not in all_stopwords]
            stop_corp = [word for word in corpus.words() if word in all_stopwords]
            counts = Counter(stop_corp)
            filtered_count = len(fil_corpus)
            top20.append(counts.most_common(20))
            results.append({'corpus': name, 'word_count': raw_count, 'word_count_without_stopwords': filtered_count, 'percentage_removed': (1 - (filtered_count / raw_count)) * 100 })
    except Exception:
        continue  # skip any that fail

# Make and display the DataFrame
df = pd.DataFrame(results).sort_values(by='word_count', ascending=False)
print(df)


                   corpus  word_count  word_count_without_stopwords  \
18                  jeita     3254399                       3250655   
15              gutenberg     2621613                       1505565   
7                comtrans     1978268                       1643819   
31                reuters     1720901                       1258365   
22          movie_reviews     1583820                        875409   
20             mac_morpho     1170095                        953771   
2                   brown     1161192                        674660   
23           multext_east     1054921                        955219   
48                  udhr2      926859                        883679   
32                 semcor      820411                        479739   
0                     abc      766811                        470377   
9               conll2002      678377                        606108   
21            masc_tagged      592536                        375761   
47    

In [95]:
df.style.format({'percentage_removed': "{:.2f}%", 'word_count_without_stopwords': "{:,}", 'word_count': "{:,}"})
print(df.to_markdown())

|    | corpus                |   word_count |   word_count_without_stopwords |   percentage_removed |
|---:|:----------------------|-------------:|-------------------------------:|---------------------:|
| 18 | jeita                 |      3254399 |                        3250655 |            0.115044  |
| 15 | gutenberg             |      2621613 |                        1505565 |           42.571     |
|  7 | comtrans              |      1978268 |                        1643819 |           16.9062    |
| 31 | reuters               |      1720901 |                        1258365 |           26.8775    |
| 22 | movie_reviews         |      1583820 |                         875409 |           44.728     |
| 20 | mac_morpho            |      1170095 |                         953771 |           18.4877    |
|  2 | brown                 |      1161192 |                         674660 |           41.8994    |
| 23 | multext_east          |      1054921 |                         955219 |    

In [92]:
print(f'number of corpora in NLTK {len(df)}')
print(f'mean percentage removed {round(df["percentage_removed"].mean(),2)}')
print(f'word count mean {round(df["word_count"].mean(),2)}')

number of corpora in NLTK 51
mean percentage removed 21.91
word count mean 476471.02


In [82]:
from collections import defaultdict
word_counts = defaultdict(int)

for corpus_top_words in top20:
    for word, count in corpus_top_words:
        word_counts[word] += count

# Convert the dictionary to a DataFrame
df_counts = pd.DataFrame(list(word_counts.items()), columns=['word', 'count'])


In [83]:
# Sort by count and display
df_counts = df_counts.sort_values(by='count', ascending=False)
# Select the head first, then apply styling
styled_df_counts = df_counts.head(20).style.format({'count': "{:,}"})
display(styled_df_counts)

,word,count
0,the,"583,580"
4,a,"347,732"
1,of,"321,415"
3,and,"293,456"
2,to,"282,264"
5,in,"217,607"
20,de,"198,037"
6,is,"124,672"
7,that,"117,080"
9,for,"93,184"


In [94]:
print(df_counts.head(20).to_markdown())

|    | word   |   count |
|---:|:-------|--------:|
|  0 | the    |  583580 |
|  4 | a      |  347732 |
|  1 | of     |  321415 |
|  3 | and    |  293456 |
|  2 | to     |  282264 |
|  5 | in     |  217607 |
| 20 | de     |  198037 |
|  6 | is     |  124672 |
|  7 | that   |  117080 |
|  9 | for    |   93184 |
| 11 | it     |   91011 |
| 41 | o      |   78672 |
| 16 | with   |   67409 |
| 35 | as     |   58344 |
| 14 | on     |   57387 |
| 12 | be     |   56757 |
| 21 | was    |   51592 |
| 19 | he     |   49513 |
| 36 | his    |   44172 |
|  8 | s      |   43418 |
